<a href="https://colab.research.google.com/github/whatiddone/MLDL_Study/blob/main/1st_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>